In [52]:
%pip install -Uq rich pandas numpy scipy 
%pip install -Uq matplotlib seaborn plotly 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
import math
from rich import print
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats as st


In [54]:
rides = pd.read_csv('rides_go.csv')
subscriptions = pd.read_csv('subscriptions_go.csv')
users = pd.read_csv('users_go.csv')

In [55]:
rides['date'] = pd.to_datetime(rides['date'])
rides.head()

,user_id,distance,duration,date
0,1,4409.919140,25.599769,2021-01-01
1,1,2617.592153,15.816871,2021-01-18
2,1,754.159807,6.232113,2021-04-20
3,1,2694.783254,18.511000,2021-08-11
4,1,4028.687306,26.265803,2021-08-28


In [56]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18068 entries, 0 to 18067
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   18068 non-null  int64         
 1   distance  18068 non-null  float64       
 2   duration  18068 non-null  float64       
 3   date      18068 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 564.8 KB


In [57]:
data = users.merge(rides, on='user_id')
data['date_month'] = data['date'].dt.month
data.head()

,user_id,name,age,city,subscription_type,distance,duration,date,date_month
0,1,Кира,22,Тюмень,ultra,4409.919140,25.599769,2021-01-01,1
1,1,Кира,22,Тюмень,ultra,2617.592153,15.816871,2021-01-18,1
2,1,Кира,22,Тюмень,ultra,754.159807,6.232113,2021-04-20,4
3,1,Кира,22,Тюмень,ultra,2694.783254,18.511000,2021-08-11,8
4,1,Кира,22,Тюмень,ultra,4028.687306,26.265803,2021-08-28,8


In [58]:
data_free = data.query('subscription_type == "free"')
data_ultra = data.query('subscription_type == "ultra"')
display(data_free.head())
data_ultra.head()

        

,user_id,name,age,city,subscription_type,distance,duration,date,date_month
6672,700,Айдар,22,Омск,free,2515.690719,14.944286,2021-01-02,1
6673,700,Айдар,22,Омск,free,846.932642,16.234663,2021-02-01,2
6674,700,Айдар,22,Омск,free,4004.434142,20.016628,2021-02-04,2
6675,700,Айдар,22,Омск,free,1205.911290,9.782872,2021-02-10,2
6676,700,Айдар,22,Омск,free,3047.379435,17.427673,2021-02-14,2


,user_id,name,age,city,subscription_type,distance,duration,date,date_month
0,1,Кира,22,Тюмень,ultra,4409.919140,25.599769,2021-01-01,1
1,1,Кира,22,Тюмень,ultra,2617.592153,15.816871,2021-01-18,1
2,1,Кира,22,Тюмень,ultra,754.159807,6.232113,2021-04-20,4
3,1,Кира,22,Тюмень,ultra,2694.783254,18.511000,2021-08-11,8
4,1,Кира,22,Тюмень,ultra,4028.687306,26.265803,2021-08-28,8


In [59]:
data_cd = data
data_cd.duration = data_cd.duration.apply(np.ceil)

In [60]:
users_pivot = pd.pivot_table(
    data_cd, 
    index=['user_id', 'date_month'],
    values=['distance', 'duration'], 
    aggfunc={
        'distance': 'sum',
        'user_id': 'count',
        'duration': 'sum',
    }
)
users_pivot.columns = ['month_distance', 'month_duration', 'month_rides']
users_pivot['subscription_type'] = users_pivot.apply(lambda x: users[users.user_id == x.name[0]].iloc[0][-1], axis=1)
users_pivot.head(10)

C:\Users\misha\AppData\Local\Temp\ipykernel_16692\2410230769.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  users_pivot['subscription_type'] = users_pivot.apply(lambda x: users[users.user_id == x.name[0]].iloc[0][-1], axis=1)


month_distance  month_duration  month_rides  \
user_id date_month                                                
1       1              7027.511294            42.0            2   
        4               754.159807             7.0            1   
        8              6723.470560            46.0            2   
        10             5809.911100            32.0            2   
        11             7003.499363            56.0            3   
        12             6751.629942            28.0            2   
2       3             10187.723006            63.0            3   
        4              6164.381824            40.0            2   
        6              3255.338202            14.0            1   
        7              6780.722964            48.0            2   

                   subscription_type  
user_id date_month                    
1       1                      ultra  
        4                      ultra  
        8                      ultra  
        10                     ultra  
        11                     ultra  
        12                     ultra  
2       3                      ultra  
        4                      ultra  
        6                      ultra  
        7                      ultra

In [61]:
def monthCostValue(row):
    if row['subscription_type'] == 'ultra':
        return 199 + (6 * row['month_duration'])
    else:
        return 50 * row['month_rides'] + (8 * row['month_duration'])

users_pivot['month_cost'] = users_pivot.apply(monthCostValue, axis=1)
users_pivot.head(20)

month_distance  month_duration  month_rides  \
user_id date_month                                                
1       1              7027.511294            42.0            2   
        4               754.159807             7.0            1   
        8              6723.470560            46.0            2   
        10             5809.911100            32.0            2   
        11             7003.499363            56.0            3   
        12             6751.629942            28.0            2   
2       3             10187.723006            63.0            3   
        4              6164.381824            40.0            2   
        6              3255.338202            14.0            1   
        7              6780.722964            48.0            2   
        8               923.007439             7.0            1   
        10             2824.636598            19.0            1   
        12             3828.248580            12.0            1   
3       1              3061.777147            13.0            1   
        2              5421.152016            39.0            2   
        4              2956.651632            21.0            1   
        5              9064.026918            63.0            3   
        7             10919.595403            67.0            3   
        9              2891.012822            23.0            1   
        12             2593.381928            13.0            1   

                   subscription_type  month_cost  
user_id date_month                                
1       1                      ultra       451.0  
        4                      ultra       241.0  
        8                      ultra       475.0  
        10                     ultra       391.0  
        11                     ultra       535.0  
        12                     ultra       367.0  
2       3                      ultra       577.0  
        4                      ultra       439.0  
        6                      ultra       283.0  
        7                      ultra       487.0  
        8                      ultra       241.0  
        10                     ultra       313.0  
        12                     ultra       271.0  
3       1                      ultra       277.0  
        2                      ultra       433.0  
        4                      ultra       325.0  
        5                      ultra       577.0  
        7                      ultra       601.0  
        9                      ultra       337.0  
        12                     ultra       277.0

Будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки?

$H0$ - помесячная подписка от пользоватлей ultra = помесячная подписке от пользоватлей free \
$H1$ - помесячная подписка от пользоватлей ultra > помесячная подписке от пользоватлей free

In [62]:
alpha = 0.05
ultra_month_cost = users_pivot[users_pivot['subscription_type'] == 'ultra']['month_cost']
free_month_cost = users_pivot[users_pivot['subscription_type'] == 'free']['month_cost']

result = st.ttest_ind(ultra_month_cost, free_month_cost, alternative='greater')

print(f'p-value: {result[1]}')

if result[1] < alpha:
    print('Нулевая гипотеза откисает')
else:
    print('Пока что нулевая гипотеза не откисает')

p-value: 4.227438894541133e-29

Нулевая гипотеза откисает

Сред.знач. поездок пользователей с подпиской больше, чем у пользователей без подписки?

$H0$ - cреднее количество поездок в месяц у пользоватей с подпиской = количеству поездок в месяц пользователей без подписки

$H1$ - среднее количесво поездок в месяц у пользователей с подпиской привышает количество поездок в месяц пользователей без подписки

In [63]:
alpha = 0.05

subcribe_ultra = users_pivot[users_pivot['subscription_type'] == 'ultra']['month_rides']
subcribe_free = users_pivot[users_pivot['subscription_type'] == 'free']['month_rides']

result = st.ttest_ind(ultra_month_cost, 
                      free_month_cost, 
                      alternative = 'greater'
)

print(f'p-value: {result[1]}')

if result[1] < alpha:
    print('Нулевая гипотеза откисает')
else:
    print('Пока что нулевая гипотеза не откисает')

p-value: 4.227438894541133e-29

Нулевая гипотеза откисает

Средняя дистанция поездки "холодные" месяца отличается от "тёплых"?

$H0$ -- дистанция поездок в холодные месеца равны дистанции поездок в тёплые месеца


$H1$ -- дистанция поездок в холодные месеца меньше, чем дистанция в тёплые

In [64]:
alpha = 0.05

cold = [10, 11, 12, 1, 2 ,3]
warm = [4, 5, 6, 7, 8, 9]

cold1 = users_pivot.query('date_month in @cold')['month_distance']
warm1 = users_pivot.query('date_month in @warm')['month_distance']



results = st.ttest_ind(
    cold1,
    warm1,
    alternative='two-sided'
)
print(cold1.mean())
print(warm1.mean())
print(f'p-value: {results[1]}')

if result[1] < alpha:
    print('Нулевая гипотеза откисает')
else:
    print('Пока что нулевая гипотеза не откисает')

4945.764339316236

5020.023356959626

p-value: 0.19365908135594664

Нулевая гипотеза откисает

Сред. время езды на самокате - 30 минут?

$H0$ Длительность поездок длится 30 минут

$H1$ Длительность поезди больше или меньше 30 минут

In [65]:
alpha = 0.05
target_value = 30

result = st.ttest_1samp(
    data.duration,
    target_value,
    alternative = 'two-sided'
)

print(f'p-value: {results[1]}')

if result[1] < alpha:
    print('Нулевая гипотеза откисает')
else:
    print('Нулевая гипотеза не откисает')


p-value: 0.19365908135594664

Нулевая гипотеза откисает